# Image similarity search

指定された画像と似ている画像を検索する。

Step 1. Define AutoEncoder
* 画像のencoder として AutoEncoder を定義する。

Step 2. Lightning Module
* encoder を pytorchlightning 用に定義する。

Step 3. Training Encoder
* encoder 訓練する。
* 訓練済みモデルをファイルに保存する。

Step 4. Embedding
* 検索対象画像と検索画像を特徴ベクトルへ変換する。
* 特徴ベクトルをファイルに保存する。

Step 5. Faiss indexes
* Step 4 で作成した検索対象画像の特徴ベクトルを検索するためのインデックスを生成する。
* インデックスをファイルに保存する。

Step 6. Image similarity search
* Step 4 で作成した検索画像の特徴ベクトルを使用して、Step 5 のインデックスを検索する。

-----

pytorch:
* https://pytorch.org/

pytorchligtning:
* https://www.pytorchlightning.ai/

faiss:
* https://github.com/facebookresearch/faiss
* https://faiss.ai/

Cifar10:
* https://pytorch.org/vision/stable/datasets.html

AutoEndoder
* https://qiita.com/MuAuan/items/a062d0c245c8f4836399

## Setup for Notebook

In [ ]:
# notebook runtime
import sys

runtime = 'local'
if 'google.colab' in sys.modules:
    runtime = 'colab'
elif _dh == ['/kaggle/working']:
    runtime = 'kaggle'
runtime

In [ ]:
if runtime == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if runtime == 'colab':
    home_path = '/content/drive/MyDrive/image_similarity_search'
else:
    home_path = '/home/jovyan/image_similarity_search'

nbs_path = f'{home_path}/nbs'
datasets_path = f'{home_path}/datasets'
models_path = f'{home_path}/models'
figs_path = f'{home_path}/figs'
logs_path = f'{home_path}/logs'

In [ ]:
%cd {nbs_path}

In [ ]:
!pip install -q pytorch_lightning

In [ ]:
!pip install -q faiss-gpu

In [ ]:
!pip install -q nb-clean

## Setup

In [ ]:
import os
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import pickle
import shutil
from PIL import Image
import glob

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchsummary import summary

import torchvision
from torchvision import transforms

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
pl.seed_everything(42)

## Dataset

In [ ]:
from torchvision.datasets import CIFAR10

In [ ]:
dataset_name = 'cifar10'

In [ ]:
def clear_datasets():
    path_cifar_10_batches_py = os.path.join(datasets_path, 'cifar-10-batches-py')
    if os.path.exists(path_cifar_10_batches_py):
        shutil.rmtree(path_cifar_10_batches_py)

    path_cifar_10_python = os.path.join(datasets_path, 'cifar-10-python.tar.gz')
    if os.path.exists(path_cifar_10_python):
        os.remove(path_cifar_10_python)

In [ ]:
#clear_datasets()

In [ ]:
def cifar10_normalization():
    normalize = transforms.Normalize(
        mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
        std=[x / 255.0 for x in [63.0, 62.1, 66.7]],
    )
    return normalize

In [ ]:
def train_transforms():
    return torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        #cifar10_normalization(),
    ])

In [ ]:
def test_transforms():
    return torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        #cifar10_normalization(),
    ])

In [ ]:
class DataModule(pl.LightningDataModule):

    def __init__(self, data_dir: str, batch_size: int=32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = int(os.cpu_count() / 2)
        self.train_transform = train_transforms()
        self.test_transform = test_transforms()
        self.classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        self.num_classes = len(self.classes)
        self.dims = (3, 32, 32) # channels, width, height

    def prepare_data(self):
        # download
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None): #train, val, testデータ分割
        # Assign train/val datasets for use in dataloaders
        datasets = CIFAR10(self.data_dir, train=True, transform=self.train_transform)
        n_train = int(len(datasets) * 0.8)
        n_val = len(datasets) - n_train
        self.ds_train, self.ds_val = torch.utils.data.random_split(datasets, [n_train, n_val])
        self.ds_test = CIFAR10(self.data_dir, train=False, transform=self.test_transform)

    def train_dataloader(self):
        return DataLoader(self.ds_train, shuffle=True, drop_last=True, batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.ds_val, shuffle=False, batch_size=self.batch_size, num_workers=self.num_workers)
 
    def test_dataloader(self):
        return DataLoader(self.ds_test, shuffle=False, batch_size=self.batch_size, num_workers=self.num_workers)

In [ ]:
datamodule = DataModule(datasets_path, BATCH_SIZE)
datamodule.prepare_data()
datamodule.setup()

In [ ]:
# check data size
train_dataloader = iter(datamodule.train_dataloader())
images, labels = next(train_dataloader)
images.shape, labels.shape

## Step 1. Define AutoEncoder

画像の特徴ベクトルを抽出するためAutoEncoder を定義する。

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(256)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [ ]:
# check data size
m = Encoder()
m(torch.randn(32, 3, 32, 32)).shape

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=16,
                                 kernel_size=2, stride=2, padding=0),
            nn.ConvTranspose2d(in_channels=16, out_channels=3,
                                 kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.decoder(x)
        return x

In [ ]:
m = Decoder()
m(torch.randn(32, 256, 8, 8)).shape

## Step 2. Lightning Module

In [ ]:
encoder_name = 'autoencoder'

In [ ]:
class LitAutoEncoder(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def training_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "test")
    
    def _common_step(self, batch, batch_idx, stage: str):
        x, y = batch
        loss = F.mse_loss(x, self(x))
        self.log(f"{stage}_loss", loss, on_step=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
# model
encoder = LitAutoEncoder()
encoder = encoder.to(device)  #for gpu
summary(encoder.encoder, (3, 32, 32))
summary(encoder.decoder, (256, 8, 8))
summary(encoder, (3, 32, 32))
print(encoder)

## Step 3. Training Encoder

In [ ]:
path_encoder = f'{models_path}/{dataset_name}_{encoder_name}.ckpt'

In [ ]:
def clear_encoder():
    if os.path.exists(path_encoder):
        os.remove(path_encoder)

In [ ]:
clear_encoder()

In [ ]:
callbacks = [EarlyStopping(monitor="val_loss")]
trainer = pl.Trainer(gpus=1, callbacks=callbacks,
                logger=TensorBoardLogger(f'{logs_path}/lightning_logs/', name='f{dataset_name}_{encoder_name}'))

if not os.path.exists(path_encoder):
    #trainer = pl.Trainer(max_epochs=10, gpus=1)
    trainer.fit(encoder, datamodule)
    trainer.save_checkpoint(path_encoder)
else:
    encoder = encoder.load_from_checkpoint(path_encoder)

In [ ]:
encoder = encoder.to(device)  #for gpu
encoder.freeze()
encoder.eval()

In [ ]:
# Test
# 'test_loss': 0.003598422510549426
results = trainer.test(encoder, datamodule)

In [ ]:
# functions to show an image
def imshow(img, file=None, title=None):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.detach().numpy()
    plt.figure(figsize=(20, 10))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    if title:
        plt.title(title)
    if file:
        plt.savefig(file + '.png')
    plt.show()

In [ ]:
# Original train images
dataiter = iter(datamodule.train_dataloader())
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images, nrow=32), f'{figs_path}/{dataset_name}_{encoder_name}_train_original', title='train images')
print(' '.join('%5s' % datamodule.classes[labels[j]] for j in range(8)))

In [ ]:
# Decoded train images
images_hat = encoder(images)
imshow(torchvision.utils.make_grid(images_hat, nrow=32), f'{figs_path}/{dataset_name}_{encoder_name}_train_decoded', title='train decoded images')
print(' '.join('%5s' % datamodule.classes[labels[j]] for j in range(8)))

In [ ]:
# Original test images
dataiter = iter(datamodule.test_dataloader())
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images, nrow=32), f'{figs_path}/{dataset_name}_{encoder_name}_test_original', title='test images')
print(' '.join('%5s' % datamodule.classes[labels[j]] for j in range(8)))

In [ ]:
images_hat = encoder(images)
imshow(torchvision.utils.make_grid(images_hat, nrow=32), f'{figs_path}/{dataset_name}_{encoder_name}_test_decoded', title='test decoded images')
print(' '.join('%5s' % datamodule.classes[labels[j]] for j in range(8)))

## Step 4. Embedding

検索対象画像と検索画像を特徴ベクトルへ変換する。

In [ ]:
path_embeded_train = f'{models_path}/{dataset_name}_{encoder_name}_embeded_train.pickle'
path_embeded_test = f'{models_path}/{dataset_name}_{encoder_name}_embeded_test.pickle'

In [ ]:
def clear_embedding():
    if os.path.exists(path_embeded_train):
        os.remove(path_embeded_train)
    if os.path.exists(path_embeded_test):
        os.remove(path_embeded_test)

In [ ]:
clear_embedding()

In [ ]:
train_dataset = CIFAR10(datasets_path, train=True, download=True)
test_dataset = CIFAR10(datasets_path, train=False, download=True)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
# with preprocess(without data augumentation)
train_dataloader = DataLoader(CIFAR10(datasets_path, train=True, download=True, transform=test_transforms()),
                               shuffle=False, batch_size=32, num_workers=0)
test_dataloader = DataLoader(CIFAR10(datasets_path, train=False, download=True, transform=test_transforms()),
                               shuffle=False, batch_size=32, num_workers=0)

In [ ]:
len(train_dataloader), len(test_dataloader)

In [ ]:
def create_embedder_model(autoencoder):
    layers = list(autoencoder.encoder.children())
    #fv = nn.Sequential(nn.AdaptiveMaxPool2d(output_size=1))
    fv = nn.Sequential(nn.Flatten(), nn.Linear(in_features=256*8*8, out_features=256))
    model = nn.Sequential(*layers, *fv)
    return model

In [ ]:
embedder = create_embedder_model(encoder)
embedder

In [ ]:
img_random = torch.randn(32, 3, 32, 32)
img_emb = embedder(img_random)
img_emb.shape

In [ ]:
def get_embeded_vector(embedder, dataloader):
    vector = []
    for i, (images, labels) in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            v = embedder(images).squeeze().cpu()
        vector.extend(v.detach().numpy())

    return vector

In [ ]:
if not os.path.exists(path_embeded_train):
    train_vectors = get_embeded_vector(embedder, train_dataloader)
    print(len(train_vectors), train_vectors[0].shape)
    with open(path_embeded_train, mode='wb') as f:
        pickle.dump(train_vectors, f)

In [ ]:
if not os.path.exists(path_embeded_test):
    test_vectors = get_embeded_vector(embedder, test_dataloader)
    print(len(test_vectors))
    with open(path_embeded_test, mode='wb') as f:
        pickle.dump(test_vectors, f)

## Step 5. Faiss indexes

検索用インデックスを生成する。

In [ ]:
import faiss

In [ ]:
path_indexer = f'{models_path}/{dataset_name}_{encoder_name}_indexer.faiss'

In [ ]:
def clear_indexer():
    if os.path.exists(path_indexer):
        os.remove(path_indexer)

In [ ]:
clear_indexer()

In [ ]:
with open(path_embeded_train, mode='rb') as f:
    train_vectors = np.array(pickle.load(f))

train_vectors.shape

In [ ]:
class FlatIndexer(object):

    def __init__(self, vector_sz: int, nlist=10, path=None):
        if path and os.path.exists(path):
            index_cpu = faiss.read_index(path)
            self.indexer = faiss.index_cpu_to_all_gpus(index_cpu)
        else:
            #index_cpu = faiss.IndexFlatIP(vector_sz) # Not Work
            quantizer = faiss.IndexFlatL2(vector_sz)
            index_cpu = faiss.IndexIVFFlat(quantizer, vector_sz, nlist, faiss.METRIC_L2)
            res = faiss.StandardGpuResources()
            self.indexer = faiss.index_cpu_to_gpu(res, 0, index_cpu)

    def index_data(self, vectors):
        self.indexer.train(vectors)
        self.indexer.add(vectors)

    def search_knn(self, query_vectors: np.array, top_docs: int):
        scores, indexes = self.indexer.search(query_vectors, top_docs)
        return scores, indexes

    def save_index(self, path):
        index_cpu = faiss.index_gpu_to_cpu(self.indexer)
        faiss.write_index(index_cpu, path)

In [ ]:
if not os.path.exists(path_indexer):
    indexer = FlatIndexer(256, nlist=20)
    indexer.index_data(train_vectors)
    indexer.save_index(path_indexer)
else:
    indexer = FlatIndexer(256, path=path_indexer)

## Step 6. Image similarity search

In [ ]:
with open(path_embeded_test, mode='rb') as f:
    test_vectors = np.array(pickle.load(f))

test_vectors.shape

In [ ]:
scores, indexes = indexer.search_knn(test_vectors, 10)

In [ ]:
scores.shape, indexes.shape

In [ ]:
for i in range(0, 20):
    print(i, indexes[i])

In [ ]:
scores[0]

In [ ]:
def show_search_images(train_dataset, test_dataset, indexes, i, dataset_name, encoder_name):
    " train_dataset の中から test_dataset で指定された画像に似ている画像を検索する "
    fig, axes = plt.subplots(1, 11, figsize=(15,5))
    test_image, test_label = test_dataset[i]
    axes[0].imshow(test_image)
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    axes[0].set_title(f'Q[{i}]')

    for col, idx in enumerate(indexes[i]):
        img, label = train_dataset[idx]
        axes[col+1].set_title(f'A[{idx}]')
        axes[col+1].imshow(img)
        axes[col+1].set_xticks([])
        axes[col+1].set_yticks([])
    plt.show()
    fig.savefig(f'{figs_path}/{dataset_name}_{encoder_name}_search_images_{i}')

In [ ]:
for i in range(10):
    show_search_images(train_dataset, test_dataset, indexes, i, dataset_name, encoder_name)

In [ ]:
def merge_search_images(figs_path, dataset_name, encoder_name):
    files = glob.glob(f"{figs_path}/{dataset_name}_{encoder_name}_search_images_*.png")
    images = None
    for file in sorted(files):
        im = np.array(Image.open(file).convert('RGB'))
        h, w, c = im.shape
        im = im[120:h-130, 110:w-80, :] # trim
        if images is None:
            images = im
        else:
            images = np.append(images, im, axis=0)
    img = Image.fromarray(images)
    img.save(f"{figs_path}/{dataset_name}_{encoder_name}_search_images.png")
    return img

In [ ]:
img = merge_search_images(figs_path, dataset_name, encoder_name)
img

In [ ]:
# !nb-clean clean notebook.ipynb

![]()